# Création du pipeline RAG

In [12]:
# Imports et chargement de la clé API

import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
import pandas as pd
import numpy as np
import faiss

# Charger la clé API
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [13]:
# Charger les vecteurs et les métadonnées

embeddings = np.load("../data/embeddings/chunks_embeddings.npy")
df_meta = pd.read_csv("../data/embeddings/chunks_metadata.csv")

# Créer les documents LangChain
documents = [
    Document(
        page_content=row["text"],
        metadata={
            "chunk_id": row["chunk_id"],
            "source": row["source"],
            "page": row["page"],
            "theme": row["theme"],
            "niveau": row["niveau"],
            "type": row["type"]
        }
    )
    for _, row in df_meta.iterrows()
]


In [14]:
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Utiliser les embeddings déjà calculés
faiss_index = faiss.read_index("../data/embeddings/faiss_index.idx")

# Créer le store LangChain
folder_path = "../data/embeddings/faiss_store"

vectorstore = FAISS.load_local(
    folder_path,
    OpenAIEmbeddings(openai_api_key=openai_api_key),
    allow_dangerous_deserialization=True
)


In [15]:
# Initialiser le modèle de langage performant en terme de coût et langue française

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-4o",  # ou "gpt-3.5-turbo" pour une option plus économique
    temperature=0.7
)


C:\Users\aheij\AppData\Local\Temp\ipykernel_18872\3576733337.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [16]:
# Fonction pour construire le prompt avec le contexte et la question

def build_prompt(context, question):
    return f"""
Tu es *Complice*, un assistant bienveillant qui aide les adolescents autistes à mieux comprendre les émotions, les relations sociales et les situations du quotidien.

Voici des extraits de documents pertinents :
{context}

Question : {question}

Réponds en français, avec clarté, douceur et concision. Utilise un ton rassurant, inclusif et jamais culpabilisant. Si la réponse est incertaine, propose des pistes ou invite à en discuter avec un adulte de confiance.
"""


In [17]:
# Fonction pour effectuer une requête RAG

def rag_query(question):
    docs = vectorstore.similarity_search(question, k=5)
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = build_prompt(context, question)
    response = llm.invoke(prompt)
    return response.content




In [18]:
# Exemple d'utilisation

question = "Pourquoi certaines personnes évitent le contact visuel ?"
print(rag_query(question))


Certaines personnes, notamment celles avec un trouble du spectre de l'autisme, peuvent éviter le contact visuel pour plusieurs raisons. Pour elles, regarder quelqu'un dans les yeux peut être très intense et accablant à cause de la multitude de stimuli qu'elles doivent gérer en même temps. Cela peut rendre difficile la concentration sur la conversation. De plus, certaines personnes peuvent simplement se sentir plus à l'aise en regardant ailleurs lorsqu'elles parlent ou écoutent.

Il est important de se rappeler que le manque de contact visuel ne signifie pas que quelqu'un est impoli ou qu'il ne s'intéresse pas à vous. Chacun a sa propre façon de communiquer et de se sentir à l'aise dans les interactions sociales.

Si tu te sens concerné par cette situation, n'hésite pas à en parler avec un adulte de confiance. Ensemble, vous pourrez explorer des stratégies pour te sentir plus à l'aise dans tes interactions sociales.


In [19]:
# Fonction pour effectuer une requête RAG - avec affichage des chunks sélectionnés

def rag_query_verbose(question):
    docs = vectorstore.similarity_search(question, k=5)

    print("🔎 Chunks sélectionnés :\n")
    for i, doc in enumerate(docs, 1):
        print(f"Chunk {i} — Source : {doc.metadata.get('source', 'inconnu')}, Page : {doc.metadata.get('page', 'N/A')}")
        print(doc.page_content[:300] + "...\n---\n")

    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = build_prompt(context, question)
    response = llm.invoke(prompt)
    print("🧠 Réponse générée :\n")
    return response.content


In [20]:
# Exemple d'utilisation

question = "Pourquoi certaines personnes évitent le contact visuel ?"
print(rag_query_verbose(question))


🔎 Chunks sélectionnés :

Chunk 1 — Source : 100 idees pour enseigner les ha - Mehdi Liratni, Page : 49
IDÉE 14 REGARDE-MOI ! L’IMPORTANCE DU CONTACT VISUEL Un des premiers contacts sociaux « à distance » que l’enfant développe avec son entourage est le contact oculaire. Voyez comme le bébé, tout jeune et malgré une pauvre vision néo-natale, peut intuitivement plonger son regard dans celui de sa maman...
---

Chunk 2 — Source : 683956044-Le-Syndrome-dAsperger-Guide-Complet, Page : 356
(Willey 1999 p. 22) Par e-mail, Carolyn m’a expliqué que : Dans le cas des néons, je ne suis pas juste confrontée aux éclats de lumière, mais aussi aux clignotements. Ils créent des « ombres » dans ma vision (qui étaient très effrayantes quand j’étais jeune) et une lon- gue exposition pouvait me ren...
---

Chunk 3 — Source : 683956044-Le-Syndrome-dAsperger-Guide-Complet, Page : 112
97 Compréhension sociale et amitié sions faciales, je recommande d’expliquer pourquoi la personne évite le contact visuel : p

In [79]:
# Tester avec plusieurs questions

questions = [
    "Comment reconnaître la colère chez quelqu’un ?",
    "Que faire si je me sens exclu dans un groupe ?",
    "Pourquoi certaines personnes parlent très vite ?",
    "Est-ce grave si je préfère être seul parfois ?"
]

for i, question in enumerate(questions, 1):
    print(f"\n🟦 Question {i} : {question}\n")

    docs = vectorstore.similarity_search(question, k=5)

    print("🔎 Chunks sélectionnés :")
    for j, doc in enumerate(docs, 1):
        print(f"  Chunk {j} — Source : {doc.metadata.get('source', 'inconnu')}, Page : {doc.metadata.get('page', 'N/A')}")
        print("  " + doc.page_content[:200].replace("\n", " ") + "...\n")

    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = build_prompt(context, question)
    response = llm.invoke(prompt)

    print("🧠 Réponse générée :")
    print(response.content)
    print("\n" + "="*80 + "\n")



🟦 Question 1 : Comment reconnaître la colère chez quelqu’un ?

🔎 Chunks sélectionnés :
  Chunk 1 — Source : Comprendre_lautisme_pour_les_nuls, Page : 185
  suivante vous donne d’autres astuces pour surmonter la période de colère. » Sécurité d’abord. Une personne en phase de colère ne se contrôle pas et doit rester en sécurité. Essayez d’enlever les objet...

  Chunk 2 — Source : Comprendre_lautisme_pour_les_nuls, Page : 179
  Gérer les émotions suscitées par le syndrome d’Asperger Les personnes avec le syndrome d’Asperger ressentent les émotions au même titre que les autres, mais avec deux différences majeures : » Elles pe...

  Chunk 3 — Source : 683956044-Le-Syndrome-dAsperger-Guide-Complet, Page : 183
  168 Le guide complet du Syndrome d’Asperger l’aptitude à envisager les solutions de nature à réduire le sentiment de colère. Les sarcasmes rendront la personne Asperger plus confuse ; et l’appel aux é...

  Chunk 4 — Source : 683956044-Le-Syndrome-dAsperger-Guide-Complet, Page : 178